# Pipeline to run all files
* ~~All scripts currently output Excel files because we were working independently~~
* ~~<b>We'll be changing this so that each script passes dataframes until the end of the pipeline.</b>~~
* Our plan is to not store anything (refresh daily) but need to consider how to deal with taking in user feedback

In [1]:
#Gather Articles
import NewsAPI as news
import os

#delete any old sorted article files
#if os.path.exists("articles.csv"):
#    os.remove("articles.csv")

#inputs in order: set manual date (yes=1/no=0, no = last 24 hours of news), manual start date as "YYYY-MM-DD"(leave empty if 0), manual end date (leave empty if 0, companies to pull (6= all 19 companies)
articleDB = news.main(1,"2019-03-12","2019-03-13",6) #output is called 'NewsAPIOutput.xlsx' in Python Scripts > Data folder

Gathering articles on (Gap Inc) OR (Foot Locker) OR (L Brands) OR Macerich OR Kimco OR TJX OR CVS OR (Home Depot) OR (Best Buy) OR (Lowe's) OR Walmart OR (Target's) OR TGT OR Amazon OR Kroger OR Walgreens OR Kohl's OR (Dollar General) OR (Bed Bath and Beyond) OR Safeway from: 2019-03-12 to 2019-03-13
337


In [2]:
#Article Cleaning (must pip install tqdm first (only once) to run)
import dataClean as dc
articleDB = dc.DataClean(articleDB)

In [3]:
#Feature Selection and Binary Article Encoding
import FeatureEncoding as fe
contentBinaryMatrix = fe.encoding(0, df=articleDB, text_col='content', norm='wnLemm')
titleBinaryMatrix = fe.encoding(0, df=articleDB, text_col = 'title', norm='wnLemm')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Padmanie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
content
wnLemm
Binary Encoding
True
title
wnLemm
Binary Encoding
True


In [4]:
#Logistic Regression Classifier + Article Ranking, complete final file is called 'results_encoding.xlsx'
import logReg as lr
articleDB = lr.runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)
#articleDB = runLogReg(titleBinaryMatrix, contentBinaryMatrix, articleDB)

Index([0, 1, 'article_id', 'prediction'], dtype='object')
                                               title  \
0  Facebook’s messaging ambitions amount to much ...   
1  Facebook's messaging ambitions amount to much ...   
2  Market movers: Stocks that saw action on Wedne...   
3  The Daily 202: Six revealing quotes that speak...   
4  Stocks making the biggest moves in the premark...   

                                         description  
0  Facebook’s messaging ambitions amount to much ...  
1  Facebook is already the leader in enabling you...  
2  Market movers: Stocks that saw action on Wedne...  
3  From Pelosi to Ryan and Cheney to Carlson, eac...  
4                                                     


In [5]:
# This code extracts and ranks "tags" + counts frequency of tag mentions in articles 
"""
Inputs: 
    1) articleDB - uses column 'content'
    2) (optional) - tag type (could be 'ngrams'{unlimited}, 'bigrams'{terms with up to 2 words}, or 'unigrams'{single terms})
        - default is 'bigrams'
        - future work: add in noun phrases, named entities 
Outputs:
    1) articleDB = articleDB with appended columns "tags" and "tags_top_5"
    2) trendingTermsDB = keyterms by # article mentions
"""
import ContextExtraction as ce
articleDB, trendingTermsDB = ce.retrieveContext(articleDB)

100%|████████████████████████████████████████████████████████████████████████████████| 216/216 [01:14<00:00,  2.44it/s]


In [6]:
import EncodingforRecommender as rec
articleDB = rec.recommender(articleDB)

tifidf Encoding
bin Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 216/216 [00:00<00:00, 298.40it/s]


tf Encoding


100%|███████████████████████████████████████████████████████████████████████████████| 216/216 [00:00<00:00, 280.36it/s]


     companys  ambitions    harken  wechat    become  centerpiece   digital  \
0         0.2       0.75  0.333333     2.0  0.039216     0.333333  0.173913   
1         0.0       0.75  0.333333     2.0  0.039216     0.333333  0.173913   
2         0.1       0.00  0.000000     0.0  0.019608     0.000000  0.000000   
3         0.0       0.00  0.000000     0.0  0.058824     0.000000  0.000000   
4         0.0       0.00  0.000000     0.0  0.000000     0.000000  0.000000   
5         0.0       0.00  0.000000     0.0  0.000000     0.000000  0.000000   
6         0.0       0.00  0.000000     0.0  0.039216     0.000000  0.000000   
7         0.0       0.00  0.000000     0.0  0.019608     0.000000  0.000000   
8         0.0       0.00  0.000000     0.0  0.000000     0.000000  0.000000   
9         0.1       0.00  0.000000     0.0  0.039216     0.000000  0.000000   
10        0.0       0.00  0.000000     0.0  0.019608     0.000000  0.000000   
11        0.0       0.00  0.000000     0.0  0.000000

[216 rows x 13276 columns]


In [7]:
import frontPage as fp
frontpage = fp.FrontPage(articleDB, trendingTermsDB)

In [8]:
import pandas as pd
articles = pd.DataFrame(frontpage['articles'])
topterms= pd.DataFrame(frontpage['topterms'])
articles.columns

Index(['date', 'origContent', 'related_articles', 'source', 'tags', 'title'], dtype='object')

In [9]:
titles = pd.DataFrame(articles['tags'])

In [17]:
pd.DataFrame.to_excel(articleDB, 'Main Version.xlsx')